# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [107]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals: 
You have a target of selling 5500 units. 
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target. 
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours: 
You have both full-time and part-time employees. 
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours. 
Similarly, like sales, there could be deviations in these working hours. 

Overtime: 
Additionally, your employees might end up working overtime. 
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you. 
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities. 
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Ask for parameters

In [108]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [109]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**Sales:**

*  `S`:  Total units sold 
*  `S_dev`: Deviation from the sales target (can be positive or negative)

**Employee Hours:**

*  `FT_hours`: Total hours worked by full-time employees
*  `PT_hours`: Total hours worked by part-time employees
*  `FT_hours_dev`: Deviation in full-time employee hours (can be positive or negative)
*  `PT_hours_dev`: Deviation in part-time employee hours (can be positive or negative)

**Overtime:**

*  `FT_overtime`: Total overtime hours for full-time employees
*  `FT_overtime_dev`: Deviation in full-time overtime hours (can be positive or negative)
*  `PT_overtime`: Total overtime hours for part-time employees 
 

**Priorities (Given)**

*  `P1`: Priority factor for sales deviation
*  `P2`: Priority factor for full-time overtime deviation
*  `P3`: Priority factor for full-time employee hours deviation 
*  `P4`: Priority factor for part-time employee hours deviation 


# 2. Ask for objective

In [110]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [111]:
Markdown(response2.text)

```
Minimize:

P1 * |S_dev| + P2 * |FT_overtime_dev| + 2 * P3 * |FT_hours_dev| + P4 * |PT_hours_dev| + 3 * PT_overtime 
``` 


# 3. Ask for constraints

In [112]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [113]:
Markdown(response3.text)

## Constraints:

**Sales:**

*  `S = 5500 + S_dev`  
*  `S = 5 * FT_hours + 2 * PT_hours` 

**Employee Hours:**

*  `FT_hours = 800 + FT_hours_dev`
*  `PT_hours = 320 + PT_hours_dev`

**Overtime:**

*  `FT_overtime = 100 + FT_overtime_dev`
*  `FT_overtime <= FT_hours` 
*  `PT_overtime <= PT_hours`

**Variable Restrictions:**

*  All variables >= 0 (You cannot have negative sales, hours worked, or overtime) 


## 3. Generate the pyomo code

In [114]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [115]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if needed
P1 = 1  
P2 = 1.2
P3 = 1.1
P4 = 0.9

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.S_dev = pyo.Var(within=pyo.Reals)
model.FT_hours = pyo.Var(within=pyo.NonNegativeReals)
model.PT_hours = pyo.Var(within=pyo.NonNegativeReals)
model.FT_hours_dev = pyo.Var(within=pyo.Reals)
model.PT_hours_dev = pyo.Var(within=pyo.Reals)
model.FT_overtime = pyo.Var(within=pyo.NonNegativeReals)
model.FT_overtime_dev = pyo.Var(within=pyo.Reals)
model.PT_overtime = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(
    expr=P1 * abs(model.S_dev)
    + P2 * abs(model.FT_overtime_dev)
    + 2 * P3 * abs(model.FT_hours_dev)
    + P4 * abs(model.PT_hours_dev)
    + 3 * model.PT_overtime,
    sense=pyo.minimize,
)

# Constraints
model.sales_constraint = pyo.Constraint(expr=model.S == 5500 + model.S_dev)
model.sales_hours_constraint = pyo.Constraint(
    expr=model.S == 5 * model.FT_hours + 2 * model.PT_hours
)
model.ft_hours_constraint = pyo.Constraint(expr=model.FT_hours == 800 + model.FT_hours_dev)
model.pt_hours_constraint = pyo.Constraint(expr=model.PT_hours == 320 + model.PT_hours_dev)
model.ft_overtime_constraint = pyo.Constraint(
    expr=model.FT_overtime == 100 + model.FT_overtime_dev
)
model.ft_overtime_limit = pyo.Constraint(expr=model.FT_overtime <= model.FT_hours)
model.pt_overtime_limit = pyo.Constraint(expr=model.PT_overtime <= model.PT_hours)

# Solve the model
solver = pyo.SolverFactory("cbc")  # Use CBC solver (feel free to change)
solver.solve(model)

# Print results
print("Solution:")
print(f"Total Units Sold (S): {pyo.value(model.S)}")
print(f"Sales Deviation (S_dev): {pyo.value(model.S_dev)}")
print(f"Full-Time Hours (FT_hours): {pyo.value(model.FT_hours)}")
print(f"Part-Time Hours (PT_hours): {pyo.value(model.PT_hours)}")
print(f"Full-Time Hours Deviation (FT_hours_dev): {pyo.value(model.FT_hours_dev)}")
print(f"Part-Time Hours Deviation (PT_hours_dev): {pyo.value(model.PT_hours_dev)}")
print(f"Full-Time Overtime (FT_overtime): {pyo.value(model.FT_overtime)}")
print(
    f"Full-Time Overtime Deviation (FT_overtime_dev): {pyo.value(model.FT_overtime_dev)}"
)
print(f"Part-Time Overtime (PT_overtime): {pyo.value(model.PT_overtime)}")
```

**Explanation and Sample Data:**

1.  **Import Pyomo:** `import pyomo.environ as pyo` imports the Pyomo library.
2.  **Sample Data:** The values for `P1`, `P2`, `P3`, and `P4` are sample priorities. You should replace these with your actual priority factors.
3.  **Model Initialization:** `model = pyo.ConcreteModel()` creates a concrete Pyomo model.
4.  **Variable Definition:** Each variable (`S`, `S_dev`, `FT_hours`, etc.) is defined using `pyo.Var()`.  We specify that all variables are non-negative (`within=pyo.NonNegativeReals`) except for the deviation variables which can be positive or negative (`within=pyo.Reals`).
5.  **Objective Function:** The objective function is defined using `pyo.Objective()`. The `expr` argument represents the mathematical expression to minimize. We use the `abs()` function for absolute values.
6.  **Constraints:** Each constraint is defined using `pyo.Constraint()`. The `expr` argument defines the mathematical relationship of the constraint.
7.  **Solver:** A solver is selected (`cbc` in this case) and used to solve the model. 
8.  **Results:** Finally, the optimized values of the variables are printed.

**To run this code:**

1.  Make sure you have Pyomo and a solver (like CBC) installed.
2.  Replace the sample data with your actual data.
3.  Save the code as a `.py` file (e.g., `optimization\_model.py`).
4.  Run the code from your terminal using `python optimization_model.py`.

This code will output the optimal values for each variable, allowing you to minimize your workforce costs while considering sales targets.


## 4. Input problem data and try running the generated code

In [118]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data if needed
P1 = 4  
P2 = 3
P3 = 2
P4 = 1

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.S_dev = pyo.Var(within=pyo.Reals)
model.FT_hours = pyo.Var(within=pyo.NonNegativeReals)
model.PT_hours = pyo.Var(within=pyo.NonNegativeReals)
model.FT_hours_dev = pyo.Var(within=pyo.Reals)
model.PT_hours_dev = pyo.Var(within=pyo.Reals)
model.FT_overtime = pyo.Var(within=pyo.NonNegativeReals)
model.FT_overtime_dev = pyo.Var(within=pyo.Reals)
model.PT_overtime = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(
    expr=P1 * abs(model.S_dev)
    + P2 * abs(model.FT_overtime_dev)
    + 2 * P3 * abs(model.FT_hours_dev)
    + P4 * abs(model.PT_hours_dev)
    + 3 * model.PT_overtime,
    sense=pyo.minimize,
)

# Constraints
model.sales_constraint = pyo.Constraint(expr=model.S == 5500 + model.S_dev)
model.sales_hours_constraint = pyo.Constraint(
    expr=model.S == 5 * model.FT_hours + 2 * model.PT_hours
)
model.ft_hours_constraint = pyo.Constraint(expr=model.FT_hours == 800 + model.FT_hours_dev)
model.pt_hours_constraint = pyo.Constraint(expr=model.PT_hours == 320 + model.PT_hours_dev)
model.ft_overtime_constraint = pyo.Constraint(
    expr=model.FT_overtime == 100 + model.FT_overtime_dev
)
model.ft_overtime_limit = pyo.Constraint(expr=model.FT_overtime <= model.FT_hours)
model.pt_overtime_limit = pyo.Constraint(expr=model.PT_overtime <= model.PT_hours)

# Solve the model
solver = pyo.SolverFactory("ipopt")  # Use CBC solver (feel free to change)
solver.solve(model)

# Print results
print("Solution:")
print(f"Total Units Sold (S): {pyo.value(model.S)}")
print(f"Sales Deviation (S_dev): {pyo.value(model.S_dev)}")
print(f"Full-Time Hours (FT_hours): {pyo.value(model.FT_hours)}")
print(f"Part-Time Hours (PT_hours): {pyo.value(model.PT_hours)}")
print(f"Full-Time Hours Deviation (FT_hours_dev): {pyo.value(model.FT_hours_dev)}")
print(f"Part-Time Hours Deviation (PT_hours_dev): {pyo.value(model.PT_hours_dev)}")
print(f"Full-Time Overtime (FT_overtime): {pyo.value(model.FT_overtime)}")
print(
    f"Full-Time Overtime Deviation (FT_overtime_dev): {pyo.value(model.FT_overtime_dev)}"
)
print(f"Part-Time Overtime (PT_overtime): {pyo.value(model.PT_overtime)}")

ValueError: Cannot load a SolverResults object with bad status: error

In [120]:
# THIS CODE WAS RUN IN COLLAB WITH COUENNE! HERE ARE THE RESULTS:

# Solution:
# Total Units Sold (S): 5500.0
# Sales Deviation (S_dev): 0.0
# Full-Time Hours (FT_hours): 800.0
# Part-Time Hours (PT_hours): 750.0
# Full-Time Hours Deviation (FT_hours_dev): 0.0
# Part-Time Hours Deviation (PT_hours_dev): 430.0
# Full-Time Overtime (FT_overtime): 100.0
# Full-Time Overtime Deviation (FT_overtime_dev): 0.0
# Part-Time Overtime (PT_overtime): 0.0

SyntaxError: invalid syntax (2341199990.py, line 3)

## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [121]:
print(response.text)

## Variables:

**Sales:**

*  `S`:  Total units sold 
*  `S_dev`: Deviation from the sales target (can be positive or negative)

**Employee Hours:**

*  `FT_hours`: Total hours worked by full-time employees
*  `PT_hours`: Total hours worked by part-time employees
*  `FT_hours_dev`: Deviation in full-time employee hours (can be positive or negative)
*  `PT_hours_dev`: Deviation in part-time employee hours (can be positive or negative)

**Overtime:**

*  `FT_overtime`: Total overtime hours for full-time employees
*  `FT_overtime_dev`: Deviation in full-time overtime hours (can be positive or negative)
*  `PT_overtime`: Total overtime hours for part-time employees 
 

**Priorities (Given)**

*  `P1`: Priority factor for sales deviation
*  `P2`: Priority factor for full-time overtime deviation
*  `P3`: Priority factor for full-time employee hours deviation 
*  `P4`: Priority factor for part-time employee hours deviation 



In [122]:
print(response2.text)

```
Minimize:

P1 * |S_dev| + P2 * |FT_overtime_dev| + 2 * P3 * |FT_hours_dev| + P4 * |PT_hours_dev| + 3 * PT_overtime 
``` 



In [123]:
print(response3.text)

## Constraints:

**Sales:**

*  `S = 5500 + S_dev`  
*  `S = 5 * FT_hours + 2 * PT_hours` 

**Employee Hours:**

*  `FT_hours = 800 + FT_hours_dev`
*  `PT_hours = 320 + PT_hours_dev`

**Overtime:**

*  `FT_overtime = 100 + FT_overtime_dev`
*  `FT_overtime <= FT_hours` 
*  `PT_overtime <= PT_hours`

**Variable Restrictions:**

*  All variables >= 0 (You cannot have negative sales, hours worked, or overtime) 



In [124]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if needed
P1 = 1  
P2 = 1.2
P3 = 1.1
P4 = 0.9

# Model
model = pyo.ConcreteModel()

# Variables
model.S = pyo.Var(within=pyo.NonNegativeReals)
model.S_dev = pyo.Var(within=pyo.Reals)
model.FT_hours = pyo.Var(within=pyo.NonNegativeReals)
model.PT_hours = pyo.Var(within=pyo.NonNegativeReals)
model.FT_hours_dev = pyo.Var(within=pyo.Reals)
model.PT_hours_dev = pyo.Var(within=pyo.Reals)
model.FT_overtime = pyo.Var(within=pyo.NonNegativeReals)
model.FT_overtime_dev = pyo.Var(within=pyo.Reals)
model.PT_overtime = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(
    expr=P1 * abs(model.S_dev)
    + P2 * abs(model.FT_overtime_dev)
    + 2 * P3 * abs(model.FT_hours_dev)
    + P4 * abs(model.PT_hours_dev)
    + 3 * model.PT_overtime,
    sense=pyo.minimize,
)

# Constraints
model.sales_constraint = pyo.Constraint(expr=model.S == 5500 + model.S_dev)
model.sales_hours_constraint = py